In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install import_ipynb 
!conda install -c conda-forge import_ipynb

In [ ]:
import sys

sys.path.append('../')

from game_runner import NegotitaionGame
from eval.game_evaluator import GameEvaluator
import agents.simple_agent as simple_agent
import agents.llm_agent as llm_agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dataclasses import dataclass, field
from math import prod, sqrt
sys.path.append('/Users/gabesmithline/Desktop/caif_negotiation/')

# Import the notebook
import import_ipynb
%run '../test_game_eval.ipynb'  
import torch
from utils.offer import Offer

from prompts.make_prompt import make_prompt
from prompts.make_prompt_bargain import make_prompt_bargain
from metrics.visualizations import (
    plot_discounted_values,
    plot_offer_evolution,
    plot_negotiation_gap,
    plot_fairness
)

pathology_results = pd.DataFrame()  
import itertools
envy_results_history = {}
from eval.metrics import *




In [4]:
def calculate_discounted_value(offer, values, gamma, round_num):
    if offer is None:
        return 0
    base_value = sum(v * q for v, q in zip(values, offer))
    return base_value * (gamma ** (round_num - 1))


In [ ]:
import time
prompt_style = 'llama_3.3_70b_maximize_value_outside_offer_cot_1_test'
#prompt_style = 'openai_o1_maximize_value_outside_offer_cot_1_test'
#prompt_style = 'bargain_maximize_results_outside_offer_emphasized_with_example'
#prompt_style = 'anthropic_3_opus_maximize_value_outside_offer_cot_1' #claude sonnet is the model
#prompt_style = 'gemini_2.0_flash_maximize_value_outside_offer_cot_.8'
date = '1_21_2025'
#o1-2024-12-17
max_rounds = 3

def calculate_discounted_value(offer, values, gamma, realization_round):
    
    if offer is None:
        return 0
    base_value = sum(v * q for v, q in zip(values, offer))
    return base_value * (gamma ** (realization_round - 1))

rounds = []
p1_values = []
p2_values = []
games = 100

pathology_results = pd.DataFrame()
envy_results_history = {}

circle = 3 #TODO:to array [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] etc. to loop through each one 
for i in range(games):
    if (i + 1) % 10 == 0:
        print(f"Game {i + 1} of {games}")
        time.sleep(2 * 60)  #Sleep for 2 minutes

    rounds = []
    p1_values = []
    p2_values = []
    p1_offers = []
    p2_offers = []
    p1_offers_utility = []
    p2_offers_utility = []
    p1_kept = None
    p2_kept = None
    
    

    game = NegotitaionGame(
        player1_agent=llm_agent.LLMAgent(player_num=0),
        player2_agent=llm_agent.LLMAgent(player_num=1),
        num_items=5,
        item_value_range=[1, 101], 
        gamma=0.9,
        max_rounds=max_rounds,
        circle=circle
    ) 

    game_history = GameHistory(
        agent_1_name="Agent1",
        agent_2_name="Agent2",
        num_items=game.num_items,
        items=torch.tensor(game.items),
        agent_1_values=torch.tensor(game.player_values[0]),
        agent_2_values=torch.tensor(game.player_values[1]),
        agent_1_outside_value=game.outside_offer_values[0],
        agent_2_outside_value=game.outside_offer_values[1]
    )
   
    game_history.agent_1_offers = []
    game_history.agent_2_offers = []
    
    game.game_results = pd.DataFrame(columns=[  
        "game_num", 
        "round_num", 
        "player", 
        "discount_rate", 
        "offer", 
        "value", 
        "undiscounted_value",
        "items",
        "your_side_of_current_offer", 
        "outside_offer", 
        "outside_offer_undiscounted",
        "accepting_an_offer_worse_than_outside_offer",
        "making_an_offer_worse_for_you_than_your_outside_offer",
        "walking_away_from_an_offer_better_than_your_outside_offer",
        "offer_no_items_or_all_items", 
        "making_offer_worse_than_previous", 
        "nash_welfare",
        "proposal_proportion_player_1",
        "proposal_proportion_player_2",
        "concession_size", 
        "security_level_player_1", 
        "security_level_player_2", 
        "average_concession_size", 
        "rawlsian_welfare", 
        "gini_coefficient", 
        "utilitarian_welfare", 
        "jain_fairness_index",
        "on_pareto_frontier",
        "av"
        "mean_absolute_difference",
        "walk_type", 
    ])

    print(f"PLAYING GAME_NUM: {i}")
    if i > 0:
        #check i - 1 in pathology_results
        print("Unique games in pathology_results:", pathology_results['game_num'].unique())
        if i - 1 in pathology_results['game_num'].unique():
            print(f"Game {i - 1}  exists in pathology_results")
        else:
            print(f"Game {i - 1} does not exist in pathology_results")
            break
    
    pareto_front = compute_pareto_frontier(game.player_values[0], game.player_values[1], game.num_items, game.items, game.outside_offer_values)
    while game.in_progress:
        time.sleep(5) #Sleep for 5 seconds before taking a step
        current_step = len(game.history[0]) + len(game.history[1]) + 1
        current_round = (current_step - 1) // 2 + 1
        current_player = 1 if current_step % 2 == 1 else 2
        game.current_round = current_round
        your_side_of_current_offer = None
        '''
        PATHOLOGIES
        '''
        making_offer_worse_than_previous = False  #Only occurs in non-final rounds
        making_offer_worse_than_outside_offer = False  #Only occurs in non-final rounds
        offer_no_items_or_all_items = False  #Only occurs in non-final rounds
        accepting_an_offer_worse_than_outside_offer = False  #Only occurs in final round
        walking_away_from_an_offer_better_than_outside_offer = False  #Only occurs in final round
        walk_away_type = None

        print(f"\n{'=' * 80}")
        print(f"Game {i + 1}, Round {current_round}, Player {current_player}'s turn (Step {current_step})")
        print(f"{'=' * 80}")

        prev_offer = game.current_offer
        game.step()
        
        '''
        PATHOLOGY 5: Walking away from an offer better than your outside offer
        '''
        if "WALK" in game.players[current_player - 1].action or (
                current_player == 2 and
                game.players[current_player - 1].action == "COUNTEROFFER" and
                game.current_round == game.max_rounds):
            if current_player == 1 and p1_kept is not None: #NEED TO BE an offer to walk away
                if np.dot(game.player_values[0], p1_kept) > game.outside_offer_values[0]: #p1_kept is from the prior round, ie current offer 
                    walking_away_from_an_offer_better_than_outside_offer = True
            elif current_player == 2 and p2_kept is not None: #NEED TO BE an offer to walk away
                if np.dot(game.player_values[1], p2_kept) > game.outside_offer_values[1]: #p2_kept is from the prior round, ie current offer 
                    walking_away_from_an_offer_better_than_outside_offer = True
        '''
        PATHOLOGY 4: Accepting an offer worse than your outside offer
        '''
        if game.players[current_player - 1].action == "ACCEPT":
            if current_player == 1:
                if np.dot(game.player_values[0], p1_kept) < game.outside_offer_values[0]:
                    accepting_an_offer_worse_than_outside_offer = True
            elif current_player == 2:
                if np.dot(game.player_values[1], p2_kept) < game.outside_offer_values[1]:
                    accepting_an_offer_worse_than_outside_offer = True

        """
        CHECK IF VALID WALK OR INVALID WALK
        """
        if "INVALID WALK" in game.players[current_player - 1].action:
            walk_away_type = "INVALID"
        elif "WALK" in game.players[current_player - 1].action:
            walk_away_type = "VALID"

        
        if game.current_offer: #if there is an offer 
            if game.players[current_player - 1].action == "COUNTEROFFER":
                if current_player == 1:
                    realization_round_p1 = current_round
                    realization_round_p2 = current_round
                else:
                    realization_round_p1 = current_round + 1
                    realization_round_p2 = current_round + 1 
            else: #realization round is the current round
                realization_round_p1 = current_round
                realization_round_p2 = current_round
            #corrected value calculations
            if current_player == 1 and game.players[current_player - 1].action == "COUNTEROFFER": #TODO: CAN REMOVE EXTRA COUNTEROFFER CONDITION
                p1_kept = game.items - np.array(game.current_offer.offer)
                p2_kept = np.array(game.current_offer.offer)
            elif current_player == 2 and game.players[current_player - 1].action == "COUNTEROFFER": #TODO: CAN REMOVE EXTRA COUNTEROFFER CONDITION 
                p1_kept = np.array(game.current_offer.offer)
                p2_kept = game.items - np.array(game.current_offer.offer)

            p1_offers_utility.append(np.dot(game.player_values[0], p1_kept))
            p2_offers_utility.append(np.dot(game.player_values[1], p2_kept))

            p1_value = calculate_discounted_value(
                p1_kept, game.player_values[0], game.gamma, realization_round_p1
            )
            p2_value = calculate_discounted_value(
                p2_kept, game.player_values[1], game.gamma, realization_round_p2
            )

            rounds.append(current_round)
            p1_values.append(p1_value)
            p2_values.append(p2_value)

            if current_player == 1:
                p1_offers.append(game.current_offer.offer)
            else:
                p2_offers.append(game.current_offer.offer)
                print(f"\nRound {current_round} Values:")
                print(f"Player 1: {p1_value:.2f}")
                print(f"Player 2: {p2_value:.2f}")

            '''
            PATHOLOGY 1: Immediate offer was worse than the offer you just rejected
            '''
            if game.players[current_player - 1].action == "COUNTEROFFER":
                if (game.current_round == 1 and current_player == 2) or game.current_round > 1:
                    if current_player == 1:
                        your_side_of_previous_offer = p1_values[-2]  #P1's value from P2's previous offer
                        your_side_of_current_offer = p1_values[-1]  #P1's value from the current offer
                    else:
                        your_side_of_previous_offer = p2_values[-2]  #P2's value from P1's previous offer
                        your_side_of_current_offer = p2_values[-1]  #P2's value from the current offer
                    if your_side_of_previous_offer > your_side_of_current_offer:
                        making_offer_worse_than_previous = True
                    else:
                        making_offer_worse_than_previous = False

                '''
                PATHOLOGY 2: Immediate offer was worse than your outside offer
                '''
                if current_player == 1:
                    outside_offer = game.outside_offer_values[0] * (game.gamma ** (current_round - 1))
                    if p1_value < outside_offer:
                        making_offer_worse_than_outside_offer = True
                    else:
                        making_offer_worse_than_outside_offer = False
                elif current_player == 2:
                    outside_offer = game.outside_offer_values[1] * (game.gamma ** current_round) #discounted outside offer at next round
                    if p2_value < outside_offer:
                        making_offer_worse_than_outside_offer = True
                    else:
                        making_offer_worse_than_outside_offer = False
            else:
                making_offer_worse_than_outside_offer = False

            '''
            PATHOLOGY 3: Offer had no items or all items
            '''
            if game.players[current_player - 1].action == "COUNTEROFFER":
                if (np.array_equal(game.current_offer.offer, [0] * game.num_items) or
                        np.array_equal(game.current_offer.offer, game.items)):
                    offer_no_items_or_all_items = True
                else:
                    offer_no_items_or_all_items = False

            '''
            NASH WELFARE: CHECKED
            - Computes Nash Welfare of ACCEPT & COUNTEROFFERS
            - Computes Nash Welfare of WALK
            '''
            if game.players[current_player - 1].action in ("ACCEPT", "COUNTEROFFER"):
                nash_welfare = sqrt(prod([
                    np.dot(game.player_values[0], p1_kept),
                    np.dot(game.player_values[1], p2_kept)
                ]))
            else:
                nash_welfare = sqrt(prod([
                    game.outside_offer_values[0],
                    game.outside_offer_values[1]
                ]))

            '''
            UTILITY FOR CURRENT ROUND (two players, undiscounted)
            - Ask about this and figure out what to do with it
            - These need to be normalized by the total utility of the round as direct comparison is not correct
            - Its possible an agent just gets better draws in a round, so we need to normalize by the total utility I could have gotten in that round
            '''
            x1_prop = game.outside_offer_values[0] if "WALK" in game.players[current_player - 1].action else  np.dot(game.player_values[0], p1_kept)  / sum([np.dot(game.player_values[0], p1_kept), np.dot(game.player_values[0], game.num_items)])
            x2_prop = game.outside_offer_values[1] if "WALK" in game.players[current_player - 1].action else  np.dot(game.player_values[1], p2_kept)  / sum([np.dot(game.player_values[1], p2_kept), np.dot(game.player_values[1], game.num_items)])

            '''
            CONCESSION SIZE:
            - C_i = max(0, U(o_{i-1}) - U(o_i)) for consecutive offers by the same player
            - It's None if the player walks away or accepts, computed if a COUNTEROFFER is made
            - We'd average these values post-game to get the average concession size for model
            '''
            if game.players[current_player - 1].action == "COUNTEROFFER" and game.current_round > 1:
                if current_player == 1:
                    
                    old_value = p1_offers_utility[-1] if p1_offers_utility else np.dot(game.player_values[0], p1_kept)
                    new_value = p1_offers_utility[-1]
                    concession_size = max(0, old_value - new_value)
                    p1_offers_utility.append(new_value) 
                else:
                    old_value = p2_offers_utility[-1] if p2_offers_utility else np.dot(game.player_values[1], p2_kept)
                    new_value = np.dot(game.player_values[1], p2_kept)
                    concession_size = max(0, old_value - new_value)
                    p2_offers_utility.append(new_value)
            else:
                concession_size = None
                if game.players[current_player - 1].action == "COUNTEROFFER":
                    if current_player == 1 and not p1_offers_utility:
                        p1_offers_utility.append(np.dot(game.player_values[0], p1_kept))
                    elif current_player == 2 and not p2_offers_utility:
                        p2_offers_utility.append(np.dot(game.player_values[1], p2_kept))

            '''
            PROPORTION OF COLLECTIVE VALUE:
            P1_share_i = np.dot(game.player_values[0], p1_kept)/ (np.dot(game.player_values[0], p1_kept)+ x2)
            P2_share_i = np.dot(game.player_values[1], p2_kept)/ (np.dot(game.player_values[0], p1_kept)+ x2)
            None if walk
            '''
            if "WALK" in game.players[current_player - 1].action:
                proportion_of_value_p1 = None
                proportion_of_value_p2 = None
            else:
                denom = np.dot(game.player_values[0], p1_kept)+ np.dot(game.player_values[1], p2_kept)
                if denom > 0:
                    proportion_of_value_p1 = np.dot(game.player_values[0], p1_kept)/ denom
                    proportion_of_value_p2 = np.dot(game.player_values[1], p2_kept)/ denom
                else:
                    proportion_of_value_p1 = 0.0
                    proportion_of_value_p2 = 0.0
            '''
            PROPOSAL PROPORTIONALITY OF AVAILABLE ITEMS/UTILITY: 
            - Fraction of total possible utility for that player.
            - It's None if the player walks away.
            '''
            if "WALK" in game.players[current_player - 1].action:
                proposal_proportion_player_1_available_items = None
                proposal_proportion_player_2_available_items = None
            else:
                total_p1_available_items = np.dot(game.player_values[0], game.items)    
                total_p2_available_items = np.dot(game.player_values[1], game.items)
                if total_p1_available_items != 0:
                    proposal_proportion_player_1_available_items = np.dot(game.player_values[0], p1_kept)/ total_p1_available_items
                else:
                    proposal_proportion_player_1_available_items = 0.0
                if total_p2_available_items != 0:
                    proposal_proportion_player_2_available_items = np.dot(game.player_values[1], p2_kept)/ total_p2_available_items
                else:
                    proposal_proportion_player_2_available_items = 0.0
            '''
            UTILITARIAN WELFARE: CHECKED
            - just some of utility of all agents
            '''
            if "WALK" in game.players[current_player - 1].action:
                utilitarian_welfare = sum([game.outside_offer_values[0], game.outside_offer_values[1]])
            else:
                utilitarian_welfare = np.dot(game.player_values[0], p1_kept) + np.dot(game.player_values[1], p2_kept)
            '''
            RAWLSIAN WELFARE:
            - How well off is the worst off agent?
            '''
            if "WALK" in game.players[current_player - 1].action:
                rawlsian_welfare = min(game.outside_offer_values[0], game.outside_offer_values[1])
            else:
                rawlsian_welfare = min(np.dot(game.player_values[0], p1_kept), np.dot(game.player_values[1], p2_kept)) 
            '''
            GINI COEFFICIENT: CHECKED
            - Computed at each step of the game
            - Updated to the "classic" Gini formula for n=2 from your literature:
            Gini(A) = | x1 - x2 | / [4 × (x1 + x2)],  -> (2 (2^2) / 2)
            where x1, x2 are the final payoffs for the two agents.
            '''
            if "WALK" in game.players[current_player - 1].action:
                wealth_distribution = np.array(
                    [game.outside_offer_values[0], game.outside_offer_values[1]],
                    dtype=float
                )
            else:
                wealth_distribution = np.array(
                    [
                        np.dot(p1_kept, game.player_values[0]),
                        np.dot(p2_kept, game.player_values[1])
                    ],
                    dtype=float
                )

            total_wealth = np.sum(wealth_distribution)
            abs(game.outside_offer_values[0]- game.outside_offer_values[1]) / 4 * np.sum([game.outside_offer_values[0], game.outside_offer_values[1]])
            if total_wealth > 0:
                x1, x2 = wealth_distribution[0], wealth_distribution[1]
                numerator = abs(x1 - x2)
                denominator = 4.0 * total_wealth  # total wealth is /2 hence 2^3 / 2 = 4
                gini_coefficient = numerator / denominator
            else:
                gini_coefficient = 0.0
            '''
            VARIANCE & COEFFICIENT OF VARIATION (2-PLAYER VERSION): CHECKED
            '''
            average_2p = total_wealth / 2.0
            squared_sum = wealth_distribution[0]**2 + wealth_distribution[1]**2
            variance_value = (squared_sum / 2.0) - (average_2p**2)
            if average_2p != 0:
                coefficient_of_variation = (np.sqrt(variance_value) / average_2p) if variance_value > 0 else 0.0
            else:
                coefficient_of_variation = 0.0
            '''
            JAIN'S FAIRNESS INDEX: CHECKED
            - Computed at each step of the game
            - Jain's Fairness Index is computed at each step of the game
            '''
            jain_fairness_index = None
            if "WALK" in game.players[current_player - 1].action:
                x1 = game.outside_offer_values[0]
                x2 = game.outside_offer_values[1]
            else:
                x1 = np.dot(p1_kept, game.player_values[0])
                x2 = np.dot(p2_kept, game.player_values[1])
            numerator = (x1 + x2)**2
            denominator = 2.0 * (x1**2 + x2**2)
            if denominator != 0:
                jain_fairness_index = numerator / denominator
            else:
                jain_fairness_index = 1.0 if (x1 == 0 and x2 == 0) else 0.0

            '''
            SECURITY LEVEL: (TODO: PROBABLY NEEDS RENAMING) CHECKED
            - Computed at each step of the game 
            - Security Level is computed at each step of the game
            '''
            if "WALK" in game.players[current_player - 1].action:
                security_level_player_1 = 0
                security_level_player_2 = 0
            else:
                security_level_player_1 = max(0, game.outside_offer_values[0] - np.dot(game.player_values[0], p1_kept))
                security_level_player_2 = max(0, game.outside_offer_values[1] - np.dot(game.player_values[1], p2_kept))

            '''
            ON PARETO FRONTIER: CHECKED
            - Computed at start of the game
            - On Pareto Frontier is computed at each step of the game
            '''   
            on_pareto_frontier = False
            for vals in pareto_front:
                if np.array_equal(vals["agent1"], p1_kept) and np.array_equal(vals["agent2"], p2_kept):
                    on_pareto_frontier = True
                    break
            '''
            MEAN ABSOLUTE DIFFERENCE (MAD) between agents in every pair 
            From literature (for n=2): CHECKED
            MAD(A) = |x1 - x2| / 2
            If both x1 and x2 are zero, we set 0.0
            '''
            if "WALK" in game.players[current_player - 1].action:
                x1 = game.outside_offer_values[0]
                x2 = game.outside_offer_values[1]
            else:
                x1 = np.dot(game.player_values[0], p1_kept)
                x2 = np.dot(game.player_values[1], p2_kept)
            if x1 == 0.0 and x2 == 0.0:
                mean_absolute_difference = 0.0
            else:
                mean_absolute_difference = abs(x1 - x2) / 2.0

        
            game.game_results = pd.concat([game.game_results, pd.DataFrame([{
                "game_num": i,
                "round_num": current_round,
                "player": current_player,
                "discount_rate": game.gamma ** (current_round - 1),
                "offer": game.current_offer.offer,
                "value": p1_value if current_player == 1 else p2_value,
                "undiscounted_value": p1_value / game.gamma ** (current_round - 1) if current_player == 1 else p2_value / game.gamma ** (current_round - 1),
                "items": game.items,
                "your_side_of_current_offer": your_side_of_current_offer,
                "outside_offer": outside_offer,
                "outside_offer_undiscounted": game.outside_offer_values[current_player - 1],
                "accepting_an_offer_worse_than_outside_offer": accepting_an_offer_worse_than_outside_offer,
                "making_an_offer_worse_for_you_than_your_outside_offer": making_offer_worse_than_outside_offer,
                "walking_away_from_an_offer_better_than_your_outside_offer": walking_away_from_an_offer_better_than_outside_offer,
                "offer_no_items_or_all_items": offer_no_items_or_all_items,
                "making_offer_worse_than_previous": making_offer_worse_than_previous,
                "nash_welfare": nash_welfare, 
                "proposal_proportion_player_1_available_items": proposal_proportion_player_1_available_items, 
                "proposal_proportion_player_2_available_items": proposal_proportion_player_2_available_items,
                "concession_size": concession_size, 
                "security_level_player_1": security_level_player_1,
                "security_level_player_2": security_level_player_2,
                "average_concession_size": None, #TODO: check and compute implementation
                "utilitarian_welfare": utilitarian_welfare,
                "rawlsian_welfare": rawlsian_welfare,
                "gini_coefficient": gini_coefficient,
                "jain_fairness_index": jain_fairness_index,
                "on_pareto_frontier": on_pareto_frontier,
                "mean_absolute_difference": mean_absolute_difference,
                "walk_away_type": walk_away_type
            }])], ignore_index=True)

        '''
        FINAL POSSIBLE ROUND PRINT STATEMENTS
        '''
        if game.current_offer is not None and len(game.current_offer.offer) == game.num_items:
            game_history.add_offer(
                game.current_player,
                Offer(game.current_player, offer=torch.tensor(game.current_offer.offer))
            )
        if current_round == game.max_rounds and current_player == 2:
            if game.current_offer and game.current_offer != prev_offer:
                print(f"\nPlayer 2's final action: COUNTEROFFER {game.current_offer.offer}")
                p1_value = game.outside_offer_values[0] * (game.gamma ** (current_round - 1))
                p2_value = game.outside_offer_values[1] * (game.gamma ** (current_round - 1))
                print("\nGame ended after max rounds - both players get outside offers")
                
            elif game.current_offer == prev_offer:
                print("\nPlayer 2's final action: ACCEPT")
                p1_kept = game.items - np.array(game.current_offer.offer)
                p1_value = calculate_discounted_value(
                    p1_kept, game.player_values[0], game.gamma, current_round
                )
                p2_value = calculate_discounted_value(
                    game.current_offer.offer, game.player_values[1], game.gamma, current_round
                )
                if game.outside_offer_values[1] > np.dot(game.player_values[1], p1_offers[-1]):
                    accepting_an_offer_worse_than_outside_offer = True
            else:
                
                print("\nPlayer 2's final action: WALK")
                p1_value = game.outside_offer_values[0] * (game.gamma ** (current_round - 1))
                p2_value = game.outside_offer_values[1] * (game.gamma ** (current_round - 1))
                print("\nGame ended after max rounds - both players get outside offers")

            game.in_progress = False
            print(f"\nRound {current_round} Final Values:")
            print(f"Player 1: {p1_value:.2f}")
            print(f"Player 2: {p2_value:.2f}")
            rounds.append(current_round)
            p1_values.append(p1_value)
            p2_values.append(p2_value)
            break
        elif game.current_offer is None and current_player == 1 and "WALK" in game.players[current_player - 1].action:
            print("Player 1 walked")
            # If Player 1 makes an invalid offer
            game.game_results = pd.concat([game.game_results, pd.DataFrame([{
                "game_num": i,
                "round_num": current_round,
                "player": current_player,
                "discount_rate": game.gamma ** (current_round - 1),
                "offer": [],
                "value": None,
                "items": game.items,
                "your_side_of_current_offer": None,
                "outside_offer": None,
                "accepting_an_offer_worse_than_outside_offer": accepting_an_offer_worse_than_outside_offer,
                "making_an_offer_worse_for_you_than_your_outside_offer": making_offer_worse_than_outside_offer,
                "walking_away_from_an_offer_better_than_your_outside_offer": walking_away_from_an_offer_better_than_outside_offer,
                "offer_no_items_or_all_items": offer_no_items_or_all_items,
                "making_offer_worse_than_previous": making_offer_worse_than_previous,
                "security_level_player_1": 0, #regret is always 0
                "security_level_player_2": 0, #regret is always 0
                "average_concession_size": None, #TODO: check and compute implementation
                "nash_welfare": sqrt(prod([
                    game.outside_offer_values[0],
                    game.outside_offer_values[1]
                ])),
                "utilitarian_welfare": game.outside_offer_values[0] + game.outside_offer_values[1],
                "rawlsian_welfare": min(game.outside_offer_values[0], game.outside_offer_values[1]),
                "gini_coefficient": abs(game.outside_offer_values[0]- game.outside_offer_values[1]) / 4 * np.sum([game.outside_offer_values[0], game.outside_offer_values[1]]),
                "security_level_player_1": 0,
                "security_level_player_2": 0,
                "jain_fairness_index": None, #TODO: check and compute implementation
                "proposal_proportion_player_1": None, #TODO: check and compute implementation
                "proposal_proportion_player_2": None, #TODO: check and compute implementation
                "concession_size": None, #TODO: check and compute implementation
                "on_pareto_frontier": False, #TODO: check and compute implementation
                "mean_absolute_difference": abs(game.outside_offer_values[0] - game.outside_offer_values[1]) / 2.0,
                "walk_away_type": walk_away_type
            }])], ignore_index=True)
            game.in_progress = False


    pathology_results = pd.concat([pathology_results, game.game_results], ignore_index=True)

   
    envy_results_history[i] = game_history

pathology_results.to_csv(f'pathology_results_{date}_{games}_{prompt_style}_circle_{circle}.csv', index=False)

# pd.DataFrame.from_dict(envy_results_history, orient='index').to_csv(
#     f'envy_results_{date}_{games}_{prompt_style}.csv', index=False
# )

## Statistics of Play for Each Player Over Games

In [ ]:
# Read the CSV data
import pandas as pd
#/Users/gabesmithline/Desktop/caif_negotiation/experiments/2025_1_5/llama/basic_maximize_value_outside_offer_cot/pathology_results_1_5_2025_100_llama_3.1_405b_maximize_value_outside_offer_cot_.9.csv
df = pd.read_csv(f'../experiments/pathology_results_{date}_{games}_{prompt_style}_circle_{circle}.csv')
#df = pd.read_csv("/Users/gabesmithline/Desktop/caif_negotiation/experiments/2025_1_4/llama/basic_maximize_value_outside_offer_cot/pathology_results_1_4_2025_100_llama_3.1_405b_maximize_value_outside_offer_cot_1.csv")
def analyze_pathology_comprehensive(df, pathology_name, is_game_ending_pathology=False):
    total_games = df['game_num'].nunique()

    if is_game_ending_pathology:
        # Identify moves where the pathology occurred
        #pathology_moves = df[df[pathology_name]]
        pathology_moves = df[df[pathology_name].fillna(False)]

        # Get the first occurrence per game (since game ends after pathology)
        first_pathology_moves = pathology_moves.groupby('game_num').first().reset_index()

        # Overall count of games with the pathology
        overall_count = len(first_pathology_moves)

        print(f"\n=== {pathology_name} ===")
        print(f"\nOverall: {overall_count}/{total_games} ({(overall_count/total_games*100):.1f}%)")

        # Count per player
        player_counts = first_pathology_moves['player'].value_counts()

        for player in [1, 2]:
            player_count = player_counts.get(player, 0)
            print(f"Player {player}: {player_count}/{total_games} ({(player_count/total_games*100):.1f}%)")

        # Count per round
        round_counts = first_pathology_moves['round_num'].value_counts()
        for round_num in sorted(df['round_num'].unique()):
            round_count = round_counts.get(round_num, 0)
            print(f"Round {round_num}: {round_count}/{total_games} ({(round_count/total_games*100):.1f}%)")

    else:
        # Identify the first occurrence of a game-ending pathology per game
        game_ending_moves = df[df[['accepting_an_offer_worse_than_outside_offer',
                                   'walking_away_from_an_offer_better_than_your_outside_offer']].any(axis=1)]
        #first_game_ending_round = game_ending_moves.groupby('game_num')['round_num'].min()
        first_game_ending_indices = game_ending_moves.groupby('game_num').first().reset_index()['round_num']
        # Function to get valid moves (before any game-ending pathology occurs)
        def get_valid_moves(group):
            game_num = group['game_num'].iloc[0]
            if game_num in first_game_ending_indices.index:
                #pathology_round = first_game_ending_round.loc[game_num]
                first_pathology_index = first_game_ending_indices.loc[game_num]

                #return group[group['round_num'] < pathology_round]  # Exclude the pathology round and after
                return group[group['round_num'] < first_pathology_index]  # Exclude the pathology round and after
            else:
                return group  

        # Apply the function to get all valid moves before game-ending pathologies
        valid_moves = df.groupby('game_num').apply(get_valid_moves).reset_index(drop=True)

        total_moves = len(valid_moves)
        count = valid_moves[pathology_name].sum()

        print(f"\n=== {pathology_name} ===")
        print(f"\nOverall: {int(count)}/{total_moves} ({(count/total_moves*100):.1f}%)")

        # By Player
        for player in [1, 2]:
            player_moves = valid_moves[valid_moves['player'] == player]
            player_count = player_moves[pathology_name].sum()
            moves = len(player_moves)
            if moves > 0:
                print(f"Player {player}: {int(player_count)}/{moves} ({(player_count/moves*100):.1f}%)")

        # By Round
        for round_num in sorted(valid_moves['round_num'].unique()):
            round_moves = valid_moves[valid_moves['round_num'] == round_num]
            round_count = round_moves[pathology_name].sum()
            moves_in_round = len(round_moves)
            if moves_in_round > 0:
                print(f"Round {round_num}: {int(round_count)}/{moves_in_round} ({(round_count/moves_in_round*100):.1f}%)")

# Define pathology types
game_ending_pathologies = [
    'accepting_an_offer_worse_than_outside_offer',
    'walking_away_from_an_offer_better_than_your_outside_offer'
]

per_move_pathologies = [
    'making_an_offer_worse_for_you_than_your_outside_offer',
    'offer_no_items_or_all_items',
    'making_offer_worse_than_previous'
]

# Analyze each pathology
for pathology in game_ending_pathologies:
    analyze_pathology_comprehensive(df, pathology, is_game_ending_pathology=True)

for pathology in per_move_pathologies:
    analyze_pathology_comprehensive(df, pathology, is_game_ending_pathology=False)

## Compute Bargaining Metrics

In [ ]:
df = pd.read_csv(f'../experiments/pathology_results_{date}_{games}_{prompt_style}_circle_{circle}.csv')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def analyze_bargaining_statistics(csv_path):
    """
    Reads the CSV of game results (including welfare and bargaining stats),
    computes aggregate measures by round, and visualizes them.
    
    Args:
        csv_path (str): Path to the CSV file (e.g. "pathology_results_1_18_2025_3_openai_o1_maximize_value_outside_offer_cot_1_test.csv")
    """

    # 1. Read the CSV into a DataFrame
    df = pd.read_csv(csv_path)

    # 2. Replace direct "None" strings or actual None with np.nan
    #    so that pandas can skip them appropriately in calculations.
    df = df.replace({None: np.nan, 'None': np.nan})

    # 3. Specify the numeric columns we wish to analyze and visualize
    #    (feel free to add/remove as needed from your dataset).
    columns_of_interest = [
        "value",
        "undiscounted_value",
        "nash_welfare",
        "proposal_proportion_player_1",
        "proposal_proportion_player_2",
        "concession_size",
        "security_level_player_1",
        "security_level_player_2",
        "rawlsian_welfare",
        "gini_coefficient",
        "utilitarian_welfare",
        "jain_fairness_index",
        
    ]
    
    # 4. Convert relevant columns to numeric, coerce errors -> NaN
    for col in columns_of_interest:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # 5. Compute summary statistics by round (mean, std, etc.)
    #    This will skip NaN by default (skipna=True).
    group_by_round = df.groupby("round_num")
    summary_stats = {}
    for col in columns_of_interest:
        if col in df.columns:
            summary_stats[col] = {
                "mean": group_by_round[col].mean(),
                "std":  group_by_round[col].std(),
                "min":  group_by_round[col].min(),
                "max":  group_by_round[col].max()
            }
    
    # 6. Print out some aggregated summaries
    print("=== AGGREGATE STATISTICS BY ROUND ===")
    print("Round |  Column                           |  Mean    |  Std     |  Min     |  Max")
    print("----------------------------------------------------------------------------------")
    unique_rounds = sorted(df["round_num"].dropna().unique())
    for rnd in unique_rounds:
        for col in columns_of_interest:
            if col in df.columns:
                col_stats = summary_stats[col]
                mean_val = col_stats["mean"].get(rnd, np.nan)
                std_val  = col_stats["std"].get(rnd, np.nan)
                min_val  = col_stats["min"].get(rnd, np.nan)
                max_val  = col_stats["max"].get(rnd, np.nan)
                print(f"{rnd:<5} | {col:<32} | {mean_val:8.2f} | {std_val:8.2f} | {min_val:8.2f} | {max_val:8.2f}")
    
    # 7. Create line plots for each column vs. round_num
    #    Only plot if the column actually exists in the DataFrame.
    columns_exist = [col for col in columns_of_interest if col in df.columns]
    nrows = len(columns_exist)
    fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(8, 4 * nrows), sharex=True)
    
    # If there's only one column, axes won't be an array
    if nrows == 1:
        axes = [axes]
    
    for ax, col in zip(axes, columns_exist):
        aggregated = group_by_round[col].mean()  # skipna=True by default
        ax.plot(aggregated.index, aggregated.values, marker='o')
        ax.set_title(f"Mean {col} by Round")
        ax.set_xlabel("Round Number")
        ax.set_ylabel(col)
    
    plt.tight_layout()
    plt.show()


analyze_bargaining_statistics(f"../experiments/pathology_results_{date}_{games}_{prompt_style}_circle_{circle}.csv")



## Compute Envy Free Results By Game

In [ ]:
#Loop through envy_results_history and compute envy free results
envy_results_by_game = pd.DataFrame()
envy_results_by_game.columns = pd.DataFrame(columns=[ #Set up dataframe to store game pathology results
        "game_num", "outside_offer_consistency", "offer_increasing", "envy_free_strict", "envy_free_exclude_one_item"
    ])
for i, game in envy_results_history.items():
    print("="*60)
    print(f"Game {i + 1}:")
    evaluator = GameEvaluator(game)
    print(f"Outside offer consistency: {evaluator.evaluate_outside_offer_consistency()}")
    print(f"Offer increasing: {evaluator.evaluate_offer_increasing()}")
    print(f"Envy-free (strict): {evaluator.evaluate_envy_free()}")
    print(f"Envy-free (exclude one item): {evaluator.evaluate_envy_free(exclude_one_item=True)}")
    envy_results_by_game = pd.concat([envy_results_by_game, pd.DataFrame([{
        "game_num": i, "outside_offer_consistency": evaluator.evaluate_outside_offer_consistency(),
        "offer_increasing": evaluator.evaluate_offer_increasing(), "envy_free_strict": evaluator.evaluate_envy_free(),
        "envy_free_exclude_one_item": evaluator.evaluate_envy_free(exclude_one_item=True)
    }])], ignore_index=True)

#percentage of games that are envy free
print(f"Envy free (strict): {envy_results_by_game['envy_free_strict'].value_counts() / len(envy_results_by_game)}")
print(f"Envy free (exclude one item): {envy_results_by_game['envy_free_exclude_one_item'].value_counts() / len(envy_results_by_game)}")
print(f"Outside offer consistency: {envy_results_by_game['outside_offer_consistency'].value_counts() / len(envy_results_by_game)}")
print(f"Offer increasing: {envy_results_by_game['offer_increasing'].value_counts() / len(envy_results_by_game)}")



## Envy Free Results by Player

In [ ]:
envy_results_history_copy = envy_results_history.copy()
for i, game in envy_results_history.items():
    print("="*60)
    print(f"Game {i + 1}:")
    print(f"Game History Values:")
    for player in [0, 1]:
        player_offers = game.get_offers(player)
        player_values = game.agent_1_values if player == 0 else game.agent_2_values
        outside_value = game.agent_1_outside_value if player == 0 else game.agent_2_outside_value
        
        print(f"\nPlayer {player + 1}:")
        print(f"Outside offer value: {outside_value}")
        
        for i, offer in enumerate(player_offers):
            given_value = torch.dot(player_values, offer.offer)
            total_value = torch.dot(player_values, game.items)
            kept_value = total_value - given_value
            
            print(f"Offer {i + 1}:")
            print(f"  Offer: {offer.offer}")
            print(f"  Player values: {player_values}")
            print(f"  Value calculation:")
            print(f"    Total value: {total_value.item()} = {player_values} · {game.items}")
            print(f"    Given value: {given_value.item()} = {player_values} · {offer.offer}")
            print(f"    Kept value: {kept_value.item()} = {total_value.item()} - {given_value.item()}")
            print(f"  Value of kept items: {kept_value.item()}")

    print("\nEnvy-free evaluation details:")
    for player in [0, 1]:
        opponent = 1 - player
        player_offers = game.get_offers(player)
        opponent_offers = game.get_offers(opponent)
        player_values = game.agent_1_values if player == 0 else game.agent_2_values
        
        print(f"\nPlayer {player + 1}:")
        for i, offer in enumerate(player_offers):
            if i < len(opponent_offers):
                opp_offer = opponent_offers[i]
                player_bundle_value = torch.dot(player_values, game.items - offer.offer)
                opponent_bundle_value = torch.dot(player_values, game.items - opp_offer.offer)
                
                print(f"Round {i + 1}:")
                print(f"  Player's offer: {offer.offer}")
                print(f"  Opponent's offer: {opp_offer.offer}")
                print(f"  Player's bundle value: {player_bundle_value.item()} = {player_values} · ({game.items} - {offer.offer})")
                print(f"  Opponent's bundle value (to player): {opponent_bundle_value.item()} = {player_values} · ({game.items} - {opp_offer.offer})")
                
                if player_bundle_value >= opponent_bundle_value:
                    print("  Envy-free: Yes")
                else:
                    print("  Envy-free: No")
                    print("  Checking if removing one item makes it envy-free:")
                    for j in range(len(game_history.items)):
                        temp_opp_offer = opp_offer.offer.clone()
                        if temp_opp_offer[j] > 0:
                            temp_opp_offer[j] -= 1
                            temp_opponent_bundle_value = torch.dot(player_values, game.items - temp_opp_offer)
                            print(f"    Removing item {j + 1}: {temp_opponent_bundle_value.item()} = {player_values} · ({game.items} - {temp_opp_offer})")
                            if player_bundle_value >= temp_opponent_bundle_value:
                                print(f"    Envy-free after removing item {j + 1}: Yes")
                                break
                    else:
                        print("    Envy-free after removing any single item: No")


In [ ]:
print("Checking envy_results_history structure:")
print(f"Type: {type(envy_results_history_copy.items())}")
print(f"Length: {len(envy_results_history_copy.items())}")
if len(envy_results_history_copy.items()) > 0:
    first_game = next(iter(envy_results_history_copy.items()))[1]
    print("\nFirst game attributes:")
    print(f"Has items: {hasattr(first_game, 'items')}")
    print(f"Has agent_1_values: {hasattr(first_game, 'agent_1_values')}")
    print(f"Has agent_1_outside_value: {hasattr(first_game, 'agent_1_outside_value')}")
    print(f"Has get_offers method: {hasattr(first_game, 'get_offers')}")
    if hasattr(first_game, 'get_offers'):
        print(f"Number of offers player 1: {len(first_game.get_offers(0))}")
        print(f"Number of offers player 2: {len(first_game.get_offers(1))}")

In [ ]:
#TODO: Move this to metrics/envy_free_analysis.py
def analyze_envy_freeness_comprehensive(game_history):
    total_games = len(game_history)
    moves_data = []
    game_properties = []
    
    for game_id, game in game_history.items():
        evaluator = GameEvaluator(game)
        max_round = max(len(game.get_offers(0)), len(game.get_offers(1)))
        
        # Track game-level properties
        is_outside_consistent = evaluator.evaluate_outside_offer_consistency()
        is_offer_increasing = evaluator.evaluate_offer_increasing()
        is_final_allocation = evaluator.evaluate_envy_free()
        is_final_one_item_ef = evaluator.evaluate_envy_free(exclude_one_item=True)
        
        game_properties.append({
            'game_num': game_id,
            'rounds_played': max_round,
            'outside_consistent': is_outside_consistent,
            'offer_increasing': is_offer_increasing
        })
        
        for round_num in range(max_round):
            steps_in_round = 0
            for player in [0, 1]:
                opponent = 1 - player
                player_offers = game.get_offers(player)
                opponent_offers = game.get_offers(opponent)
                
                is_last_step = (round_num == max_round - 1 and 
                              round_num >= len(opponent_offers) - 1)
                
                if round_num < len(player_offers):
                    steps_in_round += 1
                    player_values = game.agent_1_values if player == 0 else game.agent_2_values
                    offer = player_offers[round_num]
                    
                    if round_num < len(opponent_offers):
                        opp_offer = opponent_offers[round_num]
                        player_bundle_value = torch.dot(player_values, game.items - offer.offer)
                        opponent_bundle_value = torch.dot(player_values, game.items - opp_offer.offer)
                        
                        is_strict_ef = bool(player_bundle_value >= opponent_bundle_value)
                        is_one_item_ef = False
                        
                        if not is_strict_ef:
                            for j in range(len(game.items)):
                                temp_opp_offer = opp_offer.offer.clone()
                                if temp_opp_offer[j] > 0:
                                    temp_opp_offer[j] -= 1
                                    temp_value = torch.dot(player_values, game.items - temp_opp_offer)
                                    if player_bundle_value >= temp_value:
                                        is_one_item_ef = True
                                        break
                        
                        moves_data.append({
                            'game_num': game_id,
                            'round_num': round_num + 1,
                            'step_num': steps_in_round,
                            'player': player + 1,
                            'is_strict_ef': is_strict_ef,
                            'is_one_item_ef': is_one_item_ef,
                            'is_final_allocation': is_last_step,
                            'final_strict_ef': is_final_allocation if is_last_step else False,
                            'final_one_item_ef': is_final_one_item_ef if is_last_step else False,
                            'outside_consistent': is_outside_consistent,
                            'offer_increasing': is_offer_increasing
                        })
    
    df = pd.DataFrame(moves_data)
    game_props_df = pd.DataFrame(game_properties)
    
    def print_stats(group, name):
        strict_ef_count = group['is_strict_ef'].sum()
        one_item_ef_count = group['is_one_item_ef'].sum()
        outside_consistent_count = group['outside_consistent'].sum()
        offer_increasing_count = group['offer_increasing'].sum()
        count = len(group)
        
        print(f"\n=== {name} ===")
        print(f"Count: {count}")
        print(f"Strict Envy-Free: {strict_ef_count}/{count} ({(strict_ef_count/count*100):.1f}%)")
        print(f"One-Item Envy-Free: {one_item_ef_count}/{count} ({(one_item_ef_count/count*100):.1f}%)")
        print(f"Outside Offer Consistent: {outside_consistent_count}/{count} ({(outside_consistent_count/count*100):.1f}%)")
        print(f"Offer Increasing: {offer_increasing_count}/{count} ({(offer_increasing_count/count*100):.1f}%)")
    
    print("\nGAME LENGTH STATISTICS")
    print(f"Average rounds per game: {game_props_df['rounds_played'].mean():.1f}")
    print(f"Max rounds in any game: {game_props_df['rounds_played'].max()}")
    print(f"Min rounds in any game: {game_props_df['rounds_played'].min()}")
    
    # Overall game properties
    print("\nOVERALL GAME PROPERTIES")
    print(f"Outside Offer Consistent: {game_props_df['outside_consistent'].sum()}/{total_games} ({(game_props_df['outside_consistent'].sum()/total_games*100):.1f}%)")
    print(f"Offer Increasing: {game_props_df['offer_increasing'].sum()}/{total_games} ({(game_props_df['offer_increasing'].sum()/total_games*100):.1f}%)")
    
    # Overall statistics
    print("\nOVERALL STATISTICS")
    print(f"Total Games: {total_games}")
    print(f"Total Moves: {len(df)}")
    
    # Final allocation statistics
    final_moves = df[df['is_final_allocation']]
    final_strict = final_moves.groupby('game_num')['final_strict_ef'].any()
    final_one_item = final_moves.groupby('game_num')['final_one_item_ef'].any()
    
    print("\nFINAL ALLOCATION STATISTICS")
    print(f"Strict Envy-Free: {final_strict.sum()}/{total_games} ({(final_strict.sum()/total_games*100):.1f}%)")
    print(f"One-Item Envy-Free: {final_one_item.sum()}/{total_games} ({(final_one_item.sum()/total_games*100):.1f}%)")
    
    # By Player
    for player in [1, 2]:
        player_moves = df[df['player'] == player]
        print_stats(player_moves, f"Player {player}")
    
    # By Round
    for round_num in sorted(df['round_num'].unique()):
        round_moves = df[df['round_num'] == round_num]
        print_stats(round_moves, f"Round {round_num}")
        
        # By Step within Round
        for step_num in sorted(round_moves['step_num'].unique()):
            step_moves = round_moves[round_moves['step_num'] == step_num]
            print_stats(step_moves, f"Round {round_num} - Step {step_num}")
        
        # By Player within Round
        for player in [1, 2]:
            player_round_moves = round_moves[round_moves['player'] == player]
            print_stats(player_round_moves, f"Round {round_num} - Player {player}")

    return df

envy_stats_df = analyze_envy_freeness_comprehensive(envy_results_history_copy)  
#save to csv
envy_stats_df.to_csv(f'envy_stats_df_{date}_{games}_{prompt_style}.csv', index=False)


## Statistics 

In [ ]:
# Statistics of Envy Free Results
import pandas as pd
import torch
def compute_statistics(envy_results_history):
    # Initialize dictionaries to store statistics
    game_stats = []
    round_stats = []
    player_stats = []
    
    print(f"Number of games in history: {len(envy_results_history)}")  # Debug print
    
    for game_id, game in envy_results_history.items():
        print(f"Processing game {game_id + 1}")  # Debug print
        
        # Game level statistics
        game_data = {
            'game_id': game_id + 1,
            'total_rounds': len(game.get_offers(0)),
            'player1_outside_value': game.agent_1_outside_value,
            'player2_outside_value': game.agent_2_outside_value,
            'total_items': len(game.items)
        }
        game_stats.append(game_data)
        
        # Process each player's offers and values
        for player in [0, 1]:
            player_offers = game.get_offers(player)
            player_values = game.agent_1_values if player == 0 else game.agent_2_values
            opponent = 1 - player
            opponent_offers = game.get_offers(opponent)
            
            print(f"  Player {player + 1} has {len(player_offers)} offers")  # Debug print
            
            # Player level statistics
            player_data = {
                'game_id': game_id + 1,
                'player_id': player + 1,
                'total_value': torch.dot(player_values, game.items).item(),
                'outside_value': game.agent_1_outside_value if player == 0 else game.agent_2_outside_value,
                'num_offers': len(player_offers)
            }
            player_stats.append(player_data)
            
            # Round level statistics
            for round_id, (offer, opp_offer) in enumerate(zip(player_offers, opponent_offers)):
                given_value = torch.dot(player_values, offer.offer).item()
                kept_value = torch.dot(player_values, game.items - offer.offer).item()
                opponent_bundle_value = torch.dot(player_values, game.items - opp_offer.offer).item()
                
                is_envy_free = kept_value >= opponent_bundle_value
                
                round_data = {
                    'game_id': game_id + 1,
                    'round_id': round_id + 1,
                    'player_id': player + 1,
                    'given_value': given_value,
                    'kept_value': kept_value,
                    'opponent_bundle_value': opponent_bundle_value,
                    'is_envy_free': is_envy_free
                }
                round_stats.append(round_data)
    
    print(f"\nStatistics summary:")  # Debug prints
    print(f"Number of game records: {len(game_stats)}")
    print(f"Number of player records: {len(player_stats)}")
    print(f"Number of round records: {len(round_stats)}")
    
    # Convert to DataFrames and save to CSV
    game_df = pd.DataFrame(game_stats)
    round_df = pd.DataFrame(round_stats)
    player_df = pd.DataFrame(player_stats)
    
    # Save to CSV files
    game_df.to_csv('game_statistics.csv', index=False)
    round_df.to_csv('round_statistics.csv', index=False)
    player_df.to_csv('player_statistics.csv', index=False)
    
    # Print first few rows of each DataFrame
    print("\nFirst few rows of game statistics:")
    print(game_df.head())
    print("\nFirst few rows of player statistics:")
    print(player_df.head())
    print("\nFirst few rows of round statistics:")
    print(round_df.head())
    
    return game_df, round_df, player_df

game_df, round_df, player_df = compute_statistics(envy_results_history)


In [ ]:
import sys

# Add the parent directory to sys.path
sys.path.append('../')

from game_runner import NegotitaionGame
from eval.game_evaluator import GameEvaluator
import agents.simple_agent as simple_agent
import agents.llm_agent as llm_agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dataclasses import dataclass, field

# Add the main project directory to sys.path
sys.path.append('/Users/gabesmithline/Desktop/caif_negotiation/')

# Import the notebook
import import_ipynb
%run '../test_game_eval.ipynb'  # Run the notebook directly
import torch
from utils.offer import Offer

from prompts.make_prompt import make_prompt
from metrics.visualizations import (
    plot_discounted_values,
    plot_offer_evolution,
    plot_negotiation_gap,
    plot_fairness
)

# Initialize the DataFrame to store pathology results
pathology_results = pd.DataFrame()  # Store total game pathology results
# Add COLUMNS
envy_results_history = {}

## Discounted Values Over Time

For each offer/counter-offer:
- Blue line shows what Player 1 would receive from that division of items in their offer
- Red line shows what Player 2 would receive from that division of items counter offer


So each point on the graph shows the value each player would get if that particular division of items was accepted, after applying the time discount ($\gamma^{round-1}$).



In [ ]:
plot_discounted_values(rounds, p1_values, p2_values, max_rounds)


## Offer Evolution Plot

This visualization shows how the distribution of items changes over the negotiation rounds, with a separate subplot for each item type.

### Layout
- Creates a grid of subplots (2 columns)
- One subplot per item type
- For odd number of items, removes unused subplot

### Per Item Subplot
- X-axis: Negotiation rounds
- Y-axis: Number of units offered to P2
- Blue line (○): P1's offers
- Red line (○): P2's counter-offers
- Y-axis range: 0 to maximum units available for that item
- Grid shows valid integer unit values

### Interpretation
- Shows how many units of each item type each player offered to give to P2
- P1's remaining units = total_items - offered_units
- Converging lines suggest agreement being reached
- Diverging lines suggest areas of contention

In [ ]:
plot_offer_evolution(game, rounds, p1_offers, p2_offers)

In [ ]:
def get_final_division(game):
    if game.current_offer: 
        p2_items = game.current_offer.offer  
        p1_items = [t - g for t, g in zip(game.items, p2_items)]  
        return {
            'P1': p1_items,
            'P2': p2_items
        }
    return None
evaluator = GameEvaluator(game)
outside_offer_rational = evaluator.evaluate_outside_offer_consistency()

if game.current_offer:
    final_division = get_final_division(game)
    value_gaps = [abs(p1 - p2) for p1, p2 in zip(p1_values, p2_values)] 
    fairness = [(p1/(p1+p2), p2/(p1+p2)) for p1, p2 in zip(p1_values, p2_values)]  

    print("\nBasic Metrics:")
    print(f"Final P1 Value: {p1_values[-1]:.2f}")
    print(f"Final P2 Value: {p2_values[-1]:.2f}")
    print(f"Final Value Gap: {value_gaps[-1]:.2f}")
    print(f"Final Fairness Ratio: {fairness[-1][0]:.2f} : {fairness[-1][1]:.2f}")
    print("\nFinal Division of Items:")
    if final_division:
        print(f"Total Items Available: {game.items}")
        print(f"P1 Received: {final_division['P1']}")
        print(f"P2 Received: {final_division['P2']}")
    else:
        print("No deal reached - no items were divided")

print("\nNegotiation Evaluation:")
print(f"Outside Offer Rational: {'Yes' if outside_offer_rational else 'No'}")

print(f"P1 Valuation of Each Item: {game.player_values[0]}")
print(f"P2 Valuation of Each Item: {game.player_values[1]}")

## Check Pathologies
- Check if Pathologies Occured
- Check Count in game 
- Save to csv
